<a href="https://colab.research.google.com/github/pabloperfer/CUDAexercises/blob/main/StridingMultipleDimensions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
from numba import cuda
@cuda.jit
def add_matrix_stride(A,B,C):
  j,i = cuda.grid(2)
  stride_y,stride_x = cuda.gridsize(2)

#A.shape[0]=shape rows A.shape[1]=shape of columns
  for data_i in range (i,A.shape[0],stride_x):
    for data_j in range (j,A.shape[1],stride_y):
      C[data_i][data_j]= A[data_i][data_j] 
  
  for data_i in range (i,B.shape[0],stride_x):
    for data_j in range (j,B.shape[1],stride_y):
      C[data_i][data_j]+= B[data_i][data_j] 
      

In [20]:
A = np.arange(64*64).reshape(64,64).astype(np.int32)
B = A * 2
C = np.zeros_like(A)
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.to_device(C)

blocks = (6,6)
threads_per_block = (6,6)

add_matrix_stride[blocks, threads_per_block](d_A,d_B,d_C)

In [21]:
from numpy import testing
output = d_C.copy_to_host()
print(output)
solution=A+B
testing.assert_array_equal(output,solution)


[[    0     3     6 ...   183   186   189]
 [  192   195   198 ...   375   378   381]
 [  384   387   390 ...   567   570   573]
 ...
 [11712 11715 11718 ... 11895 11898 11901]
 [11904 11907 11910 ... 12087 12090 12093]
 [12096 12099 12102 ... 12279 12282 12285]]
